# Introduction

The Monty Hall Problem is something of a cultural phenomenon. The problem reveals that our intuition can often fail us on even simple problems. Many often struggle to understand the solution even after a thorough explanation, and a intermediate understanding of statistics is often needed. In this notebook, I hope to demonstrate the solution in a more practical and accessible way.

# Demonstration

In [119]:
import random

In [120]:
def setting_up_the_doors():
    # decide the car door
    door_list = ["goat", "goat", "goat"]
    chosen_door = random.randint(0,2)
    door_list[chosen_door] = "car"
    return door_list   

In [121]:
def choose_a_door(door_list):
    chosen_door = random.randint(0,2)
    door_contains = door_list[chosen_door]

    return chosen_door

In [122]:
def reveal_goat(door_list, chosen_door):
    doors_with_goats = get_doors_with_goats(door_list)
    revealed_door = random.choice(doors_with_goats)
       
    if revealed_door == chosen_door:
        doors_with_goats.remove(revealed_door)
        revealed_door = doors_with_goats[0]
    
    return revealed_door

In [123]:
def get_doors_with_goats(door_list):
    doors_with_goats = [i for i, x in enumerate(door_list) if x == "goat"]
    return doors_with_goats

In [124]:
def switch(door_list, chosen_door, revealed_doors):
    doors_with_goats = get_doors_with_goats(door_list)
    door_numbers = [0,1,2]
    door_numbers.remove(chosen_door)
    door_numbers.remove(revealed_door)
    
    chosen_door = door_numbers[0]
    
    return chosen_door

In [125]:
def access_win(door_list, chosen_door):
    win = 0
    if (door_list[chosen_door]=='car'):
        win = 1
        
    return win

## Game Setup

In [126]:
doors = setting_up_the_doors()
chosen_door = choose_a_door(doors)
print("Door No.{} was chosen".format(chosen_door+1))
revealed_door = reveal_goat(doors, chosen_door)
print("Door No.{} was revealed to have a goat".format(revealed_door+1))

Door No.2 was chosen
Door No.1 was revealed to have a goat


## Staying

In [127]:
print("You chose to stay with your choice")
if access_win(doors, chosen_door) == 1:
    print("You win")
else:
    print ("you lose")

You chose to stay with your choice
You win


## Switching

In [128]:
print("You chose to switch your choice")
switch_choice = switch(doors, chosen_door, revealed_door)
if access_win(doors, switch_choice) == 1:
    print("You win")
else:
    print ("you lose")

You chose to switch your choice
you lose


# Test

Now let's show the solution by running many Monty Hall Problems

In [145]:
numbers_of_games = 100
stay_win = 0
switch_win = 0

for x in range(numbers_of_games):
    
    
    doors = setting_up_the_doors()
    chosen_door = choose_a_door(doors)
    revealed_door = reveal_goat(doors, chosen_door)
    
    if access_win(doors, chosen_door) == 1:
        stay_win += 1
    else:
        switch_win += 1

print("The results are staying: {}% and switching: {}%".format(round(stay_win/numbers_of_games*100), round(switch_win/numbers_of_games*100)))      

The results are staying: 34% and switching: 66%


These results prove the solution